## 1. Boston and San Francisco Airbnb - Data Wrangling

### Imports

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
import datetime as datetime
import math as math
import csv

### Functions

In [2]:
# Functions
def count_nulls (dataframe):
    """
    Count the nulls in the dataset
    
    dataframe - dataframe to count nulls in
    """
    return np.sum(dataframe.isna().sum())


def find_data_if_not_null(dataframe,columnName):
    """
    Show content of column where not null useful to see if there is anything there at all
    
    dataframe - pd.DataFrame to process
    columnName - Name of column to show
    
    """
    return dataframe[columnName][~dataframe[columnName].isnull()]


def drop_all_null(dataframe):
    """
    Find columns that are all Null/NaN and drop
    
    dataframe - pd.DataFrame to process
    """
    to_drop=[]
    for col in dataframe.columns:
        if( len(find_data_if_not_null(dataframe,col)) == 0):
            print(col)
            to_drop.append(col)
    return dataframe.drop(to_drop, axis=1)

def print_datatypes(dataframe):
    """
    Print Datatypes of columns in a dataframe
    
    dataframe - pd.DataFrame to process
    """
    for x in dataframe.columns:
        print(dataframe[x].name, " : ", dataframe[x].dtype)
        
def remove_number_format(dataframe,columnName):
    """
    Remove currency formatting. Remove '$' and ',' from number strings that are currency 
    and convert to float data type.
    
    dataframe - pd.DataFrame to process
    columnName - column name of column to process
    """
    dataframe[columnName] = dataframe[columnName].fillna('0.0')
    dataframe[columnName] = dataframe[columnName].replace({'\$':''}, regex = True)
    dataframe[columnName] = dataframe[columnName].replace({',':''}, regex = True)
    dataframe[columnName] = dataframe[columnName].astype(float)
    
def replace_null_to_empty(dataframe,columnName):
    """
    Replace null in column with empty string
    
    dataframe - dataframe to process
    columnname - column name of column to process
    """
    dataframe[columnName] = dataframe[columnName].fillna('')
    
def replace_null_to_Unknown(dataframe,columnName):
    """
    Replace null in column with 'Unknown' string
    """
    dataframe[columnName] = dataframe[columnName].fillna('Unknown')
    
def fix_percentage_string(dataframe,columnName):
    """
    Replace % sign with empty string and change type  to float 
    
    dataframe - pd.DataFrame to process
    columnName - column name of column to process
    """
    dataframe[columnName] = dataframe[columnName].fillna('0.0')
    dataframe[columnName] = dataframe[columnName].replace({'%':''}, regex = True)
    dataframe[columnName] = dataframe[columnName].astype(float)
    
def replace_null_to_zero_float(dataframe,columnName):
    """
    Replace null in column with 0.0 float
    
    dataframe - pd.DataFrame to process
    columnName - column name of column to process
    """
    dataframe[columnName] = dataframe[columnName].fillna('0.0')
    dataframe[columnName] = dataframe[columnName].astype(float)
    
    
def replace_string_zero(dataframe,columnName):
    """
    Replace empty string with zero
    
    dataframe - pd.DataFrame to process
    columnName - column name of column to process
    """
    dataframe[columnName] = dataframe[columnName].replace({'':'0'},regex=True)
    
def replace_date(dataframe,columnName):
    """
    Fill in nulls with default date 
    
    dataframe - pd.DataFrame to process
    columnName - column name of column to process
    """
    dataframe[columnName] = dataframe[columnName].fillna('1900-01-01')
    dataframe[columnName] = dataframe[columnName].astype(str)
    dataframe[columnName] =  pd.to_datetime(dataframe[columnName].stack(), format='%Y-%m-%d', errors='raise').unstack()
        
def count_nulls (dataframe):
    """
    Count the nulls in the dataset
    
    dataframe - dataframe to count nulls in
    """
    return np.sum(dataframe.isna().sum())


def fix_boolean(dataframe, columnName):
    """
    Replace tf with True False
    
        dataframe - pd.DataFrame to process
    columnName - column name of column to process
    """
    dataframe[columnName] = dataframe[columnName].fillna('f')
    dataframe[columnName] = dataframe[columnName].replace('f','', regex=True)
    dataframe[columnName] = dataframe[columnName].replace('t','True', regex=True)
    dataframe[columnName] = dataframe[columnName].astype(bool)
def data_to_csv(dataframe, fname):
    dataframe.to_csv(path_or_buf=fname, sep=';', na_rep='', float_format=None, columns=None,
          header=True, index=False, index_label="Index", mode='w', encoding='utf-8', compression='infer',
          quoting=csv.QUOTE_NONNUMERIC, quotechar='"', line_terminator=None, chunksize=None, date_format=None,
          doublequote=True, escapechar=None, decimal='.', errors='strict', storage_options=None)


### Business Understanding 
Airbnb are a online accomodation service, that provide a platform for hosts to advertise property for rent. Airbnb do not provide data for research purposes. However the <a href="insideairbnb.com"> insideairbnb project</a> has collected data from publicly available sources. See the insideairbnb.com site for further details regarding the use of this data and the way that it is gathered.

### Data Exploration

The data is available <a href="http://insideairbnb.com/get-the-data.html">here at insideairbnb.com</a> and is licensed under a <a href="http://creativecommons.org/publicdomain/zero/1.0/" >Creative Commons CC0 1.0 Universal (CC0 1.0) "Public Domain Dedication" license.</a>  
The data is the result in part if not all, of user data entry. This means that fields may be partially completed or will contain whatever values the user thought were good at the time. You may see the summary field filled out with "see below" presumably referencing the description field. Similarly some of the numeric fields might benefit from imputation.

A  <a href="https://docs.google.com/spreadsheets/d/1iWCNJcSutYqpULSQHlNyGInUvHg2BoUGoNRIGa6Szc4/edit#gid=982310896"> data dictionary</a> of the data is available <a href="https://docs.google.com/spreadsheets/d/1iWCNJcSutYqpULSQHlNyGInUvHg2BoUGoNRIGa6Szc4/edit#gid=982310896">  here</a>

### Data Load

In [3]:
# Load data from CSV file

boston_url='https://github.com/hj32/Notebooks/raw/main/airbnb/Boston_Listings.csv'
boston_datafile="boston_listings.csv"
san_francisco_url='https://github.com/hj32/Notebooks/raw/main/airbnb/San_Francisco_Listings.csv'
san_francisco_datafile="san_francisco_listings.csv"

#bdf=pd.read_csv(boston_url)
#sfdf=pd.read_csv(san_francisco_url)

bdf=pd.read_csv(boston_datafile, encoding='utf_8')
sfdf=pd.read_csv(san_francisco_datafile, encoding='utf_8')

print("Boston Data ", bdf.shape)

print("San Francisco", sfdf.shape)

Boston Data  (3146, 74)
San Francisco (6682, 74)


In [4]:
# Count Nulls in datasets
print("Boston Null Count",count_nulls(bdf))
print("San Francisco Null Count",count_nulls(sfdf))

Boston Null Count 27787
San Francisco Null Count 56588


There are lots of nulls to fill in so lets begin.

In [5]:
# Find columns that are all Null/NaN and drop
print("-- Boston --")
bdf = drop_all_null(bdf)
print("-- San Francisco --")
sfdf = drop_all_null(sfdf)

-- Boston --
neighbourhood_group_cleansed
bathrooms
calendar_updated
-- San Francisco --
neighbourhood_group_cleansed
bathrooms
calendar_updated


In [6]:
# Check Datatypes
print_datatypes(bdf)

id  :  int64
listing_url  :  object
scrape_id  :  int64
last_scraped  :  object
name  :  object
description  :  object
neighborhood_overview  :  object
picture_url  :  object
host_id  :  int64
host_url  :  object
host_name  :  object
host_since  :  object
host_location  :  object
host_about  :  object
host_response_time  :  object
host_response_rate  :  object
host_acceptance_rate  :  object
host_is_superhost  :  object
host_thumbnail_url  :  object
host_picture_url  :  object
host_neighbourhood  :  object
host_listings_count  :  float64
host_total_listings_count  :  float64
host_verifications  :  object
host_has_profile_pic  :  object
host_identity_verified  :  object
neighbourhood  :  object
neighbourhood_cleansed  :  object
latitude  :  float64
longitude  :  float64
property_type  :  object
room_type  :  object
accommodates  :  int64
bathrooms_text  :  object
bedrooms  :  float64
beds  :  float64
amenities  :  object
price  :  object
minimum_nights  :  int64
maximum_nights  :  int64

In [7]:
# Check Datatypes
print_datatypes(sfdf)

id  :  int64
listing_url  :  object
scrape_id  :  int64
last_scraped  :  object
name  :  object
description  :  object
neighborhood_overview  :  object
picture_url  :  object
host_id  :  int64
host_url  :  object
host_name  :  object
host_since  :  object
host_location  :  object
host_about  :  object
host_response_time  :  object
host_response_rate  :  object
host_acceptance_rate  :  object
host_is_superhost  :  object
host_thumbnail_url  :  object
host_picture_url  :  object
host_neighbourhood  :  object
host_listings_count  :  float64
host_total_listings_count  :  float64
host_verifications  :  object
host_has_profile_pic  :  object
host_identity_verified  :  object
neighbourhood  :  object
neighbourhood_cleansed  :  object
latitude  :  float64
longitude  :  float64
property_type  :  object
room_type  :  object
accommodates  :  int64
bathrooms_text  :  object
bedrooms  :  float64
beds  :  float64
amenities  :  object
price  :  object
minimum_nights  :  int64
maximum_nights  :  int64

In [8]:
# Find columns with nulls
bdf.isna().sum()[bdf.isna().sum() > 0]

description                      35
neighborhood_overview          1014
host_name                        78
host_since                       78
host_location                    81
host_about                     1145
host_response_time              694
host_response_rate              694
host_acceptance_rate            680
host_is_superhost                78
host_thumbnail_url               78
host_picture_url                 78
host_neighbourhood              273
host_listings_count              78
host_total_listings_count        78
host_has_profile_pic             78
host_identity_verified           78
neighbourhood                  1014
bathrooms_text                    5
bedrooms                        413
beds                            100
minimum_minimum_nights            2
maximum_minimum_nights            2
minimum_maximum_nights            2
maximum_maximum_nights            2
minimum_nights_avg_ntm            2
maximum_nights_avg_ntm            2
first_review                

In [9]:
# find columns with nulls
sfdf.isna().sum()[sfdf.isna().sum()>0]

description                      73
neighborhood_overview          1905
host_name                       341
host_since                      341
host_location                   346
host_about                     2161
host_response_time             1805
host_response_rate             1805
host_acceptance_rate           1675
host_is_superhost               341
host_thumbnail_url              341
host_picture_url                341
host_neighbourhood              764
host_listings_count             341
host_total_listings_count       341
host_has_profile_pic            341
host_identity_verified          341
neighbourhood                  1905
bathrooms_text                    6
bedrooms                        882
beds                             48
minimum_minimum_nights            1
maximum_minimum_nights            1
minimum_maximum_nights            1
maximum_maximum_nights            1
minimum_nights_avg_ntm            1
maximum_nights_avg_ntm            1
first_review                

### Data Processing
I am going to impute values to fill in some fields where they contain nulls

### Tidy datasets - remove nulls

In [10]:
bdf[bdf['property_type'].isna()]

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month


In [11]:
#bdf[bdf['host_listings_count'].isna()].head(5)

#bdf['host_listings_count'].unique()
bdf[bdf['beds']==0].head(3)

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,3781,https://www.airbnb.com/rooms/3781,20210420194355,2021-04-21,HARBORSIDE-Walk to subway,Fully separate apartment in a two apartment bu...,"Mostly quiet ( no loud music, no crowed sidewa...",https://a0.muscache.com/pictures/24670/b2de044...,4804,https://www.airbnb.com/users/show/4804,Frank,2008-12-03,Massachusetts,My wife and I and grown children frequently oc...,within a day,100%,41%,f,https://a0.muscache.com/im/users/4804/profile_...,https://a0.muscache.com/im/users/4804/profile_...,East Boston,5.0,5.0,"['email', 'phone', 'reviews']",t,f,"Boston, Massachusetts, United States",East Boston,42.36413,-71.02991,Entire apartment,Entire home/apt,2,1 bath,1.0,0.0,"[""Washer"", ""Dishwasher"", ""Refrigerator"", ""Pati...",$125.00,32,1125,32.0,32.0,1125.0,1125.0,32.0,1125.0,t,1,1,1,106,2021-04-21,19,3,0,2015-07-10,2021-02-26,99.0,10.0,10.0,10.0,10.0,10.0,10.0,NaN,f,1,1,0,0,0.27
4,13247,https://www.airbnb.com/rooms/13247,20210420194355,2021-04-21,Back Bay studio apartment,Beautiful studio apartment in a great Back Bay...,The Back Bay has so many great shops & restaur...,https://a0.muscache.com/pictures/miso/Hosting-...,51637,https://www.airbnb.com/users/show/51637,Susan,2009-11-05,"Boston, Massachusetts, United States",Real Estate Professional,NaN,NaN,NaN,f,https://a0.muscache.com/im/users/51637/profile...,https://a0.muscache.com/im/users/51637/profile...,Back Bay,0.0,0.0,"['email', 'phone']",t,f,"Boston, Massachusetts, United States",Back Bay,42.35164,-71.08752,Entire home/apt,Entire home/apt,1,1 bath,NaN,0.0,"[""Washer"", ""Dishwasher"", ""Fire extinguisher"", ...",$75.00,91,365,91.0,91.0,365.0,365.0,91.0,365.0,f,0,0,0,0,2021-04-21,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,2,2,0,0,NaN
5,16384,https://www.airbnb.com/rooms/16384,20210420194355,2021-04-21,Small Room in Cambridge Kendall MIT,<b>The space</b><br />No Frills Accommodations...,NaN,https://a0.muscache.com/pictures/506512/5d8f63...,23078,https://www.airbnb.com/users/show/23078,Eric,2009-06-24,"Boston, Massachusetts, United States",Hello! Professional in 30s splitting time mos...,within an hour,80%,20%,f,https://a0.muscache.com/im/users/23078/profile...,https://a0.muscache.com/im/users/23078/profile...,Beacon Hill,5.0,5.0,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,t,NaN,Beacon Hill,42.35956,-71.06980,Private room in apartment,Private room,1,1 shared bath,1.0,0.0,"[""Wifi"", ""Long term stays allowed""]",$50.00,91,730,91.0,91.0,730.0,730.0,91.0,730.0,t,30,60,90,365,2021-04-21,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,1,0,1,0,NaN


In [12]:
# CleanUp Boston Data

# Cleanup string fields with null's and replace with empty string
replace_null_to_empty(bdf,['bathrooms_text'])

bdf=bdf.drop(['description','neighborhood_overview','host_name','host_location','host_about','host_response_time',
'host_thumbnail_url','host_picture_url','host_neighbourhood','neighbourhood','license'], axis=1)

# Cleanup ###% fields
fix_percentage_string(bdf,['host_response_rate', 'host_acceptance_rate'])

# Cleanup Address field
replace_null_to_empty(bdf,['property_type'])

# Cleanup bathroom bedroom beds
replace_null_to_zero_float(bdf,['bedrooms', 'beds','review_scores_rating', 'review_scores_accuracy','review_scores_cleanliness',
                           'review_scores_checkin', 'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'reviews_per_month','host_listings_count','host_total_listings_count','minimum_minimum_nights',
                          'maximum_minimum_nights','minimum_maximum_nights','maximum_maximum_nights','minimum_nights_avg_ntm',
                          'maximum_nights_avg_ntm'])

# Cleanup First Review Date  yyyy-mm-dd
replace_date(bdf, ['last_scraped','first_review','last_review','host_since','calendar_last_scraped'])

# Clean up the price field
remove_number_format(bdf, ['price'])

replace_string_zero(bdf,['bathrooms_text'])

# Clean up boolean values
fix_boolean (bdf, ['host_is_superhost','host_has_profile_pic','host_identity_verified','instant_bookable','has_availability'])

In [13]:
# CleanUp San Francisco Data

sfdf=sfdf.drop(['description','neighborhood_overview','host_name','host_location','host_about','host_response_time',
'host_thumbnail_url','host_picture_url','host_neighbourhood','neighbourhood','license'], axis=1)

# Cleanup string fields with nulls and replace with empty string
replace_null_to_empty(sfdf,['bathrooms_text'])
# Cleanup ###% fields
fix_percentage_string(sfdf,['host_response_rate', 'host_acceptance_rate'])

# Cleanup bathroom bedroom beds
replace_null_to_zero_float(sfdf,[ 'bedrooms', 'beds','review_scores_rating', 'review_scores_accuracy','review_scores_cleanliness',
                           'review_scores_checkin', 'review_scores_communication', 'review_scores_location',
                           'review_scores_value', 'reviews_per_month','host_listings_count','host_total_listings_count',
                           'minimum_minimum_nights', 'maximum_minimum_nights','minimum_maximum_nights','maximum_maximum_nights','minimum_nights_avg_ntm',
                           'maximum_nights_avg_ntm'])

# Cleanup First Review Date  yyyy-mm-dd
replace_date(sfdf, ['last_scraped','first_review','last_review','host_since','calendar_last_scraped'])

# Clean up the price field
remove_number_format(sfdf, ['price'])

replace_string_zero(sfdf,['bathrooms_text'])

# CleanUp boolean values
fix_boolean (sfdf, ['host_is_superhost','host_has_profile_pic','host_identity_verified','instant_bookable','has_availability'])

In [14]:
# No more nulls!
# Count Nulls in datasets
print("Boston Null Count",count_nulls(bdf))
print("San Francisco Null Count",count_nulls(sfdf))

Boston Null Count 0
San Francisco Null Count 0


#### Zero Prices
Drop records with zero prices, there are a limited number listings with price 0$, this seems like an error or partially completed listing and throws errors in log calculations.

In [15]:
# Drop rows with 'price'==0
bdf=bdf.drop(bdf[bdf.price==0].index, axis=0)
sfdf=sfdf.drop(sfdf[sfdf.price==0].index, axis=0)

In [16]:
print(np.sum(bdf.isnull().sum()))
print(np.sum(sfdf.isnull().sum()))

0
0


### Export dataframe to CSV file
I am going to export the cleaned up data for Boston and San Francisco so I can make some maps using the geo data and publish them on tableau public. I have used log price to colour the price charts because the price distribution is skewed to the down size.

In [17]:
# Add name of dataset so I can make a union for the Tableau workbook
bdf['Dataset']="Boston"
sfdf['Dataset']="San Francisco"


# Export the data to two csv files to upload to tableau public. I'm going to comment it out as only need to run it once
# Export dataframe for tableau

data_to_csv(bdf,"Boston_airbnb_listings_tidied.csv")
data_to_csv(sfdf,"San_Francisco_airbnb_listings_tidied.csv")